# Инициализация пакетов

In [24]:
import numpy as np
import pandas as pd

from datetime import datetime

In [25]:
#df = pd.read_excel('data/sample.xlsx')
df = pd.read_excel('data/with_coordinates.csv')

# Добавление новы переменных

In [26]:
# добавление переменной повторное обращение
df['repeated'] = df['Вид выполненных работ'].map(lambda i: 'повторное обращение' in i.lower())
# добавление переменной невыполненных работ
df['unfinished'] = df['Отзыв'].astype('str').map(lambda i: 'работы не выполнены' in i.lower())

In [24]:
# оценка жителей
# признак инцидента
# травм. опасные случаи
# Отзыв
# Оценка качества выполнения работ
# Код оценки качества выполнения ра


# Разница по времени в минутах между началом и закрытием

In [27]:
def count_time_dif(strart_date, end_date):
    return (end_date - strart_date).total_seconds()/60

created_date = df['Дата создания заявки в формате Timezone']
beginin_date = df['Дата начала действия версии заявки в формате Timezone']
closure_date = df['Дата закрытия']

# removing endings
created_date = created_date.map(lambda t_str: t_str[:t_str.index(".")])
beginin_date = beginin_date.map(lambda t_str: t_str[:t_str.index(".")])
closure_date = closure_date.map(lambda t_str: t_str[:t_str.index(".")])

# encoding to datetime
created_date = created_date.map(lambda x: datetime.strptime(x, "%Y-%m-%d %H:%M:%S")) 
beginin_date = beginin_date.map(lambda x: datetime.strptime(x, "%Y-%m-%d %H:%M:%S")) 
closure_date = closure_date.map(lambda x: datetime.strptime(x, "%Y-%m-%d %H:%M:%S")) 

df['Дата создания заявки в формате Timezone'] = created_date
df['Дата начала действия версии заявки в формате Timezone'] = beginin_date
df['Дата закрытия'] = closure_date

In [ ]:
# подсчёт разницы во времени
df['created_time_diff'] = df.apply(lambda x: count_time_dif(x['Дата создания заявки в формате Timezone'], x['Дата закрытия']), axis=1)
df['beginin_time_diff'] = df.apply(lambda x: count_time_dif(x['Дата начала действия версии заявки в формате Timezone'], x['Дата закрытия']), axis=1)

# Критерии анамальности

In [67]:
def find_close_request(df, bnd_time_dif = 72*60):
    created = False
    res = pd.DataFrame(columns = df.columns)
    for i in range(df.shape[0]):
        for j in range(i):
            open_idx = df.columns.get_loc('Дата создания заявки в формате Timezone')
            close_idx = df.columns.get_loc('Дата закрытия')
            time_dif = count_time_dif(df.iloc[j, open_idx], df.iloc[i, close_idx])
            if time_dif < bnd_time_dif:
                if not created:
                    res = df.append(df.iloc[j, :], ignore_index=True)
                    created = True
                else:
                    res = df.append(df.iloc[j, :], ignore_index=True)
    if created:
        return res

def get_incidents(df, get_filter_mask):
  res = df[get_filter_mask(df)]
  res = df.groupby(["Адрес проблемы", "Наименование дефекта"]).filter(lambda x: len(x)>1)
  res = res.groupby(["Адрес проблемы", "Наименование дефекта"], group_keys=False).apply(lambda x: find_close_request(x))
  return res

## 1 Заявки закрытые без выполнения

статус: закрытые/закрытые через МАРМ

тип дефектов все кроме: 
    
    - Ввод в эксплуатацию ИПУ воды (замена, демонтаж, пропуск межповерочного интервала),
    - Подача документов о поверке ИПУ воды в электронном виде

результативность кроме: Выполнено

результат закрытия кроме: Аварийное/плановое отключение

тип дефекта один и тот же

In [59]:
def get_filter_mask_1(df):
  active_col = df['Наименование статуса заявки'].astype('str')
  mask = active_col.map(lambda i: 'закрыт' in i.lower())

  active_col = df['Наименование дефекта'].astype('str')
  mask = np.logical_and( mask, 
                        active_col.map(lambda i:  not ( ('ввод в эксплуатацию ипу воды' in i.lower()) or 
                                                        ('подача документов о поверке' in i.lower()) ) )
                      )

  active_col = df['Результативность']
  mask = np.logical_and( mask, 
                        active_col.map(lambda i: 'Выполнено' not in i)
                      )

  active_col = df['Вид выполненных работ']
  mask = np.logical_and( mask, 
                        active_col.map(lambda i: 'Аварийное/плановое отключение' not in i)
                      )
  return mask

In [68]:
anom_1 = get_incidents(df, get_filter_mask_1)
anom_1.shape

/tmp/ipykernel_7671/1467709850.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  res = df.append(df.iloc[j, :], ignore_index=True)


(39, 73)

In [10]:
anom_1.to_csv('data/incidents_1.csv')

## 2 Заявки в статусе "закрытые" но на деле нет

статус: закрытые/закрытые через МАРМ

тип дефектов все кроме: 
    
    - Ввод в эксплуатацию ИПУ воды (замена, демонтаж, пропуск межповерочного интервала),
    - Подача документов о поверке ИПУ воды в электронном виде

результативность: Выполнено

отсутствует: Кол-во возвратов на доработку

срок выполнения: >10

один и тот же адресс и разница между временем заявки (Наименование дефекта)

In [64]:
def get_filter_mask_2(df):
  active_col = df['Наименование статуса заявки'].astype('str')
  mask = active_col.map(lambda i: 'закрыт' in i.lower())

  active_col = df['Наименование дефекта'].astype('str')
  mask = np.logical_and( mask, 
                        active_col.map(lambda i:  not ( ('ввод в эксплуатацию ипу воды' in i.lower()) or 
                                                        ('подача документов о поверке' in i.lower()) ) )
                      )

  active_col = df['Результативность']
  mask = np.logical_and( mask, 
                        active_col.map(lambda i: 'Выполнено' in i)
                      )

  active_col = df['created_time_diff']
  mask = np.logical_and(mask,
          active_col.map(lambda i: i>10))

  return mask

def get_incidents_2(df):
  res = df[get_filter_mask_2(df)]
  res = df.groupby(["Адрес проблемы", "Наименование дефекта"]).filter(lambda x: len(x)>1)
  res = res.groupby(["Адрес проблемы", "Наименование дефекта"], group_keys=False).apply(lambda x: find_close_request(x))
  return res


In [69]:
anom_2 = get_incidents(df, get_filter_mask_2)
anom_2.shape

/tmp/ipykernel_7671/1467709850.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  res = df.append(df.iloc[j, :], ignore_index=True)


(39, 73)

## 3 Быстрые невыполненные заявки
статус: закрытые/закрытые через МАРМ

результативность: Выполнено

отсутствует: Кол-во возвратов на доработку

срок выполнения: <10

при этом работа требует меньше 10 (из приложения 2)

один и тот же адресс и разница между временем заявки (Наименование дефекта)

In [71]:
def get_filter_mask_3(df):
  active_col = df['Наименование статуса заявки'].astype('str')
  mask = active_col.map(lambda i: 'закрыт' in i.lower())

  active_col = df['Результативность']
  mask = np.logical_and( mask, 
                        active_col.map(lambda i: 'Выполнено' in i))

  active_col = df['Кол-во возвратов на доработку']
  mask = np.logical_and(mask,
          active_col.map(lambda i: np.isnan(i)))

  active_col = df['created_time_diff']
  mask = np.logical_and(mask,
          active_col.map(lambda i: i<10))
  
# вставить приложение два

  return mask

In [72]:
anom_3 = get_incidents(df, get_filter_mask_3)
anom_3.shape

/tmp/ipykernel_7671/1467709850.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  res = df.append(df.iloc[j, :], ignore_index=True)


(39, 73)

## 4 Слишком быстрые заявки

результативность: Выполнено

срок выполнения: <10
при этом работа требует больше 10

количество возвратов на доработку: пусто


In [75]:
def get_filter_mask_4(df):
  active_col = df['Результативность']
  mask = active_col.map(lambda i: 'Выполнено' in i)

  active_col = df['Кол-во возвратов на доработку']
  mask = np.logical_and(mask,
          active_col.map(lambda i: np.isnan(i)))

  active_col = df['created_time_diff']
  mask = np.logical_and(mask,
          active_col.map(lambda i: i<10))
  
# вставить приложение про больше 10 мин

  return mask

In [76]:
anom_4 = get_incidents(df, get_filter_mask_4)
anom_4.shape

/tmp/ipykernel_7671/1467709850.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  res = df.append(df.iloc[j, :], ignore_index=True)


(39, 73)

## 5 возвращённые заявки

статус: закрытые/закрытые через МАРМ

Кол-во возвратов на доработку: не пусто и не 0

Отобрать наименование дефектов из приложения 4

Результативность: выполнено

In [82]:
def get_filter_mask_5(df):
  active_col = df['Наименование статуса заявки'].astype('str')
  mask = active_col.map(lambda i: 'закрыт' in i.lower())

  active_col = df['Результативность']
  mask = np.logical_and( mask, 
                        active_col.map(lambda i: 'Выполнено' in i))

  active_col = df['Кол-во возвратов на доработку']
  mask = np.logical_and(mask,
          active_col.map(lambda i: not (np.isnan(i) or i==0) ))

# наименование дефектов из приложения 4

  return mask

In [83]:
anom_5 = get_incidents(df, get_filter_mask_5)
anom_5.shape

/tmp/ipykernel_7671/1467709850.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  res = df.append(df.iloc[j, :], ignore_index=True)


(39, 73)

## соединение пунктов

In [99]:
bad_mes = pd.concat([anom_1, anom_2, anom_3, anom_4, anom_5])
bad_mes.drop_duplicates(inplace=True)

In [101]:
bad_mes.head(1)

,Корневой ИД заявки,ИД версии заявки,Номер заявки,Уникальный номер обращения ГУ (mos.ru),Дата создания заявки в формате Timezone,Дата начала действия версии заявки в формате Timezone,Наименование источника поступления заявки,Код источника поступления заявки,Имя создателя заявки,Признак инцидента,...,Оценка качества выполнения работ,Код оценки качества выполнения ра,Наименование категории платности,Код категории платности,Признак оплаты картой,repeated,unfinished,created_time_diff,beginin_time_diff,coordinates
0,96254972,96299229,04472363/21,NaN,2021-09-20 17:33:56,2021-09-20 20:02:15,телефон,phone,voxcom,Нет,...,NaN,NaN,бесплатная,free,Нет,False,False,148.316667,0.0,NaN


In [102]:
df['color'] = np.zeros(df.shape[0])
def encode_anomalies(row):
    if row['Корневой ИД заявки'] in bad_mes['Корневой ИД заявки']:
        return 2
    else:
        return 0
df['color'].map(encode_anomalies)

TypeError: 'float' object is not subscriptable

# Считывание координат

In [17]:
geo_data = pd.read_csv('data_sanya.csv', sep='|', skiprows=1, low_memory=False)

In [19]:
geo_data = geo_data[['Упрощённое написание адреса или описание местоположения', 'geodata_center']]
geo_data.dropna(axis='index', inplace=True)
geo_data.head()

,Упрощённое написание адреса или описание местоположения,geodata_center
0,"Косинская улица, дом 26А","{""coordinates"":[37.8281893944955,55.7174827851..."
8,"улица Панфилова, дом 11, строение 1","{""coordinates"":[37.4918057069316,55.8077713279..."
26,"9-я Парковая улица, дом 43/26","{""coordinates"":[37.7984137523718,55.8022031297..."
34,"Фрязевская улица, дом 9, корпус 1","{""coordinates"":[37.822531843712,55.74627944075..."
45,"улица Верхние Поля, дом 26А","{""coordinates"":[37.7539178129885,55.6661694512..."


In [53]:
def get_coordinates(adress):
    coord = geo_data[geo_data.iloc[:,0] == adress]['geodata_center']
    try:
        coord = coord.iloc[0]
        return coord[coord.index("[")+1: coord.index("]")]
    except:
        return np.nan

df['coordinates'] = df['Адрес проблемы'].map(get_coordinates)


In [54]:
df.to_csv('data/with_coordinates.csv', index=False, sep = '$')